In [1]:
from deeplotx import TextBinaryClassifierTrainer, LongTextEncoder

long_text_encoder = LongTextEncoder(
    max_length=2048,
    chunk_size=448,
    overlapping=32
)

trainer = TextBinaryClassifierTrainer(
    long_text_encoder=long_text_encoder,
    batch_size=2,
    train_ratio=0.9
)

[DEBUG] 2025-07-23 17:26:25,196 deeplotx.embedding : BertEncoder initialized on device: cpu.


In [2]:
from deeplotx.util import get_files, read_file

pos_data_path = './data/pos'
neg_data_path = './data/neg'
pos_data = [read_file(x) for x in get_files(pos_data_path)]
neg_data = [read_file(x) for x in get_files(neg_data_path)]
model = trainer.train(pos_data, neg_data, 
                      num_epochs=36, learning_rate=2e-5, 
                      balancing_dataset=True, alpha=1e-4, 
                      rho=.2, hidden_dim=256, 
                      recursive_layers=2)

[DEBUG] 2025-07-23 17:26:25,286 deeplotx.embedding : Embedding "#章节 01某天喝多，我把不苟言笑，禁欲系的老板当成了白月光。一觉醒来，我紧张问：「昨天……我们……」他答：「怎么，江秘书不打算负责？」我觉得我要死了，肯定要被炒鱿鱼了，可老板却说放我一周的假让我好好休息休息。一周后他把我堵墙角，问道：「给了你一周时间，想...".
Token indices sequence length is longer than the specified maximum sequence length for this model (24694 > 512). Running this sequence through the model will result in indexing errors
[DEBUG] 2025-07-23 17:26:29,016 deeplotx.embedding : Embedding "《一介替身》人人嫉妒我只是一介替身，却能稳坐探花郎夫人之位。闻言我内心苦涩，表面仍旧操持着偌大的沈府。我意外失去孩子的那日，沈尧瑾还在花楼之上搂着美娇娘风花雪月。我主动送去了那纸和离书。「你不过是月娘的替身罢了，若不是你有几分像她，我怎么会娶你？」「替身就...".
[DEBUG] 2025-07-23 17:26:32,991 deeplotx.trainer : Epoch 1/36 | Train Loss: 4.8806 | Valid Loss: 4.6721
[DEBUG] 2025-07-23 17:26:33,120 deeplotx.trainer : Epoch 2/36 | Train Loss: 4.7995
[DEBUG] 2025-07-23 17:26:33,244 deeplotx.trainer : Epoch 3/36 | Train Loss: 4.7731
[DEBUG] 2025-07-23 17:26:33,398 deeplotx.trainer : Epoch 4/36 | Train Loss: 4.7475 | Valid Loss: 4.7384
[DEBUG] 2025-07-23 17:26:33,503 deeplotx.train

In [3]:
model.save(model_name='test', model_dir='model')

LongContextRecursiveSequential(
  (lstm): LSTM(768, 256, num_layers=2, batch_first=True, bidirectional=True)
  (regressive_head): LinearRegression(
    (fc1): Linear(in_features=512, out_features=1024, bias=True)
    (fc1_to_fc4_res): Linear(in_features=1024, out_features=64, bias=True)
    (fc2): Linear(in_features=1024, out_features=768, bias=True)
    (fc3): Linear(in_features=768, out_features=128, bias=True)
    (fc4): Linear(in_features=128, out_features=64, bias=True)
    (fc5): Linear(in_features=64, out_features=1, bias=True)
    (parametric_relu_1): PReLU(num_parameters=1)
    (parametric_relu_2): PReLU(num_parameters=1)
    (parametric_relu_3): PReLU(num_parameters=1)
    (parametric_relu_4): PReLU(num_parameters=1)
  )
  (self_attention): SelfAttention(
    (q_proj): Linear(in_features=768, out_features=768, bias=True)
    (k_proj): Linear(in_features=768, out_features=768, bias=True)
    (v_proj): Linear(in_features=768, out_features=768, bias=True)
  )
  (proj): Linear(in

In [4]:
model = model.load(model_name='test', model_dir='model')
model.predict(long_text_encoder.encode('这是一个测试文本.', flatten=False))

[DEBUG] 2025-07-23 17:26:37,931 deeplotx.embedding : Embedding "这是一个测试文本.".


tensor([[-0.1495]])